In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def load_data(file_path):
    return pd.read_csv(file_path)

def preprocess_dates(df):
    df['date'] = pd.to_datetime(df['date'])
    df['expiration'] = pd.to_datetime(df['expiration'])
    df['hv_year_high_date'] = pd.to_datetime(df['hv_year_high_date'])
    df['hv_year_low_date'] = pd.to_datetime(df['hv_year_low_date'])
    df['iv_year_high_date'] = pd.to_datetime(df['iv_year_high_date'])
    df['iv_year_low_date'] = pd.to_datetime(df['iv_year_low_date'])
    return df

def calculate_relative_strike(df):
    df['relative_strike'] = (df['strike'] - df['current_close']) / df['current_close'] * 100
    return df

def calculate_relative_spreads(df):
    df['relative_spread_call'] = (df['ask_call'] - df['bid_call']) / df['mid_call'] * 100
    df['relative_spread_put'] = (df['ask_put'] - df['bid_put']) / df['mid_put'] * 100
    return df

def calculate_hv_iv_lengths(df):
    df['hv_year_high_length_ago'] = (df['date'] - df['hv_year_high_date']).dt.days / 365.25
    df['hv_year_low_date_length_ago'] = (df['date'] - df['hv_year_low_date']).dt.days / 365.25
    df['iv_year_high_length_ago'] = (df['date'] - df['iv_year_high_date']).dt.days / 365.25
    df['iv_year_low_date_length_ago'] = (df['date'] - df['iv_year_low_date']).dt.days / 365.25
    return df

def one_hot_encode_month(df):
    df['month'] = df['date'].dt.month
    month_dummies = pd.get_dummies(df['month'], prefix='month', drop_first=True)
    df = df.drop(columns=['month'])
    df = pd.concat([df, month_dummies], axis=1)
    return df

def create_final_df(df):
    features = [
        "4_week_option", "relative_strike", "mid_call", "relative_spread_call", "mid_put", "relative_spread_put",
        "vol_call", "delta_call", "gamma_call", "theta_call", "vega_call", "rho_call",
        "vol_put", "delta_put", "gamma_put", "theta_put", "vega_put", "rho_put",
        "hv_current", "hv_week_ago", "hv_month_ago", "hv_year_high", "hv_year_low",
        "hv_year_high_length_ago", "hv_year_low_date_length_ago",
        "iv_current", "iv_week_ago", "iv_month_ago", "iv_year_high", "iv_year_low",
        "iv_year_high_length_ago", "iv_year_low_date_length_ago"
    ]
    month_columns = [col for col in df.columns if col.startswith('month_')]
    features += month_columns
    target = "delta_neutral_long_straddle_returns"
    final_df = df[['date'] + features + [target, 'mean_daily_delta_neutral_long_straddle_returns']]
    return final_df

def normalize_features(df):
    columns_to_normalize = df.columns.difference([col for col in df.columns if col.startswith(('date', 'month_', 'delta_neutral_long_straddle_returns', 'mean_daily_delta_neutral_long_straddle_returns'))])
    scaler = MinMaxScaler()
    df.loc[:, columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
    return df

def save_final_df(df, file_path):
    df.to_csv(file_path, index=False)

def document_columns():
    doc = {
        "date": "Date of the option data.",
        "4_week_option": "Dummy variable that is 0 when the option is 2 weeks and 1 when it is 4 weeks.",
        "relative_strike": "Percent difference of the 'strike' price relative to 'current_close'.",
        "mid_call": "Midpoint of 'bid_call' and 'ask_call'.",
        "relative_spread_call": "Percent spread relative to 'mid_call' using 'bid_call' and 'ask_call'.",
        "mid_put": "Midpoint of 'bid_put' and 'ask_put'.",
        "relative_spread_put": "Percent spread relative to 'mid_put' using 'bid_put' and 'ask_put'.",
        "vol_call": "Volatility of call option.",
        "delta_call": "Delta of call option.",
        "gamma_call": "Gamma of call option.",
        "theta_call": "Theta of call option.",
        "vega_call": "Vega of call option.",
        "rho_call": "Rho of call option.",
        "vol_put": "Volatility of put option.",
        "delta_put": "Delta of put option.",
        "gamma_put": "Gamma of put option.",
        "theta_put": "Theta of put option.",
        "vega_put": "Vega of put option.",
        "rho_put": "Rho of put option.",
        "hv_current": "Current historical volatility.",
        "hv_week_ago": "Historical volatility a week ago.",
        "hv_month_ago": "Historical volatility a month ago.",
        "hv_year_high": "Highest historical volatility over the past year.",
        "hv_year_low": "Lowest historical volatility over the past year.",
        "hv_year_high_length_ago": "Fraction of a year difference between 'date' and 'hv_year_high_date'.",
        "hv_year_low_date_length_ago": "Fraction of a year difference between 'date' and 'hv_year_low_date'.",
        "iv_current": "Current implied volatility.",
        "iv_week_ago": "Implied volatility a week ago.",
        "iv_month_ago": "Implied volatility a month ago.",
        "iv_year_high": "Highest implied volatility over the past year.",
        "iv_year_low": "Lowest implied volatility over the past year.",
        "iv_year_high_length_ago": "Fraction of a year difference between 'date' and 'iv_year_high_date'.",
        "iv_year_low_date_length_ago": "Fraction of a year difference between 'date' and 'iv_year_low_date'.",
        "mean_daily_delta_neutral_long_straddle_returns": "Daily average of delta-neutral long straddle returns.",
    }
    doc.update({f"month_{i}": f"One-hot encoded column for month {i}." for i in range(2, 13)})
    doc["delta_neutral_long_straddle_returns"] = "Target variable representing the delta-neutral long straddle returns."
    return doc

def save_documentation_text(doc, file_path):
    with open(file_path, 'w') as f:
        for column, description in doc.items():
            f.write(f"{column}: {description}\n")

# Load and preprocess data
file_path = 'MSFT_final_straddle_data.csv'
df = load_data(file_path)
df = preprocess_dates(df)

# Feature engineering
df['4_week_option'] = (df['days_to_expiration'] == 28).astype(int)
df['mean_daily_delta_neutral_long_straddle_returns'] = df['delta_neutral_long_straddle_returns'] / df['days_to_expiration']
df = calculate_relative_strike(df)
df = calculate_relative_spreads(df)
df = calculate_hv_iv_lengths(df)
df = one_hot_encode_month(df)

# Create final DataFrame
final_df_raw = create_final_df(df)

# Save the raw final DataFrame
output_file_raw = 'final_straddle_data_raw.csv'
save_final_df(final_df_raw, output_file_raw)

# Normalize features
final_df_normalized = normalize_features(final_df_raw.copy())

# Save the normalized final DataFrame
output_file_normalized = 'final_straddle_data_normalized.csv'
save_final_df(final_df_normalized, output_file_normalized)

# Documentation
documentation = document_columns()

# Save documentation to a text file
documentation_file = 'final_straddle_data_documentation.txt'
save_documentation_text(documentation, documentation_file)








In [52]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Raw DataFrame:")
final_df_raw.head(100)

Raw DataFrame:


,date,4_week_option,relative_strike,mid_call,relative_spread_call,mid_put,relative_spread_put,vol_call,delta_call,gamma_call,theta_call,vega_call,rho_call,vol_put,delta_put,gamma_put,theta_put,vega_put,rho_put,hv_current,hv_week_ago,hv_month_ago,hv_year_high,hv_year_low,hv_year_high_length_ago,hv_year_low_date_length_ago,iv_current,iv_week_ago,iv_month_ago,iv_year_high,iv_year_low,iv_year_high_length_ago,iv_year_low_date_length_ago,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,delta_neutral_long_straddle_returns,mean_daily_delta_neutral_long_straddle_returns
0,2020-01-10,0,-10.127679,16.625,14.736842,0.100,80.000000,0.2407,0.9894,0.0037,-0.0143,0.0089,0.0549,0.2329,-0.0087,0.0032,-0.0061,0.0074,-0.0005,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-1.875000,-0.133929
1,2020-01-10,0,-3.929588,6.625,12.830189,0.425,11.764706,0.1518,0.9171,0.0319,-0.0328,0.0482,0.0542,0.1924,-0.1356,0.0360,-0.0464,0.0699,-0.0079,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,1.396392,0.099742
2,2020-01-10,0,-2.380065,4.725,3.174603,0.790,12.658228,0.1782,0.7663,0.0544,-0.0672,0.0968,0.0456,0.1788,-0.2348,0.0547,-0.0606,0.0996,-0.0135,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,1.059744,0.075696
3,2020-01-10,0,3.818026,0.305,16.393443,6.775,21.402214,0.1602,0.1234,0.0403,-0.0378,0.0644,0.0075,0.2123,-0.8109,0.0416,-0.0604,0.0893,-0.0362,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-29.325500,-2.094679
4,2020-01-10,0,-7.028633,11.625,6.451613,0.160,25.000000,0.2407,0.9434,0.0150,-0.0375,0.0360,0.0539,0.2329,-0.0510,0.0142,-0.0272,0.0331,-0.0032,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-0.397828,-0.028416
5,2020-01-10,0,6.917072,0.065,15.384615,11.050,23.529412,0.1784,0.0301,0.0121,-0.0140,0.0216,0.0018,0.1784,-0.9809,0.0131,-0.0070,0.0172,-0.0122,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-1.232894,-0.088064
6,2020-01-10,0,2.268504,0.725,12.413793,4.300,4.651163,0.1589,0.2469,0.0629,-0.0585,0.0997,0.0150,0.1594,-0.7574,0.0639,-0.0520,0.1009,-0.0346,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-61.615090,-4.401078
7,2020-01-10,0,0.718981,1.495,8.695652,2.450,21.224490,0.1564,0.4218,0.0792,-0.0722,0.1236,0.0255,0.1475,-0.5866,0.0848,-0.0613,0.1230,-0.0301,0.1269,0.1154,0.1024,0.4051,0.0915,0.000000,0.076660,0.2210,0.2215,0.1673,0.3298,0.1459,0.988364,0.156057,False,False,False,False,False,False,False,False,False,False,False,-20.158358,-1.439883
8,2020-01-17,0,-10.233396,17.425,15.781923,0.220,18.181818,0.3404,0.9518,0.0090,-0.0464,0.0327,0.0543,0.3378,-0.0469,0.0089,-0.0384,0.0321,-0.0031,0.1306,0.1269,0.0946,0.3978,0.0915,0.000000,0.095825,0.2190,0.2210,0.1545,0.3128,0.1459,0.292950,0.175222,False,False,False,False,False,False,False,False,False,False,False,-1.401711,-0.100122
9,2020-01-17,0,-7.241176,12.175,15.195072,0.460,17.391304,0.3038,0.9040,0.0171,-0.0669,0.0557,0.0530,0.3038,-0.0958,0.0172,-0.0598,0.0566,-0.0060,0.1306,0.1269,0.0946,0.3978,0.0915,0.000000,0.095825,0.2190,0.2210,0.1545,0.3128,0.1459,0.292950,0.175222,False,False,False,False,False,False,False,False,False,False,False,0.517120,0.036937


In [53]:
print("\nNormalized DataFrame:")
final_df_normalized.head(50)


Normalized DataFrame:


,date,4_week_option,relative_strike,mid_call,relative_spread_call,mid_put,relative_spread_put,vol_call,delta_call,gamma_call,theta_call,vega_call,rho_call,vol_put,delta_put,gamma_put,theta_put,vega_put,rho_put,hv_current,hv_week_ago,hv_month_ago,hv_year_high,hv_year_low,hv_year_high_length_ago,hv_year_low_date_length_ago,iv_current,iv_week_ago,iv_month_ago,iv_year_high,iv_year_low,iv_year_high_length_ago,iv_year_low_date_length_ago,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,delta_neutral_long_straddle_returns,mean_daily_delta_neutral_long_straddle_returns
0,2020-01-10,0,0.445624,0.150751,0.074301,0.000811,0.402858,0.114385,0.9894,0.046717,0.966114,0.021835,0.230672,0.095260,0.9913,0.037736,0.985053,0.018222,0.998349,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-1.875000,-0.133929
1,2020-01-10,0,0.527144,0.060019,0.064688,0.003912,0.058107,0.035496,0.9171,0.402778,0.922275,0.118253,0.227731,0.058829,0.8644,0.424528,0.886302,0.172125,0.973919,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,1.396392,0.099742
2,2020-01-10,0,0.547524,0.042780,0.016006,0.007394,0.062621,0.058923,0.7663,0.686869,0.840758,0.237488,0.191597,0.046595,0.7652,0.645047,0.851507,0.245260,0.955431,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,1.059744,0.075696
3,2020-01-10,0,0.629044,0.002677,0.082653,0.064498,0.106799,0.042950,0.1234,0.508838,0.910427,0.157998,0.031513,0.076729,0.1891,0.490566,0.851997,0.219897,0.880489,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-29.325500,-2.094679
4,2020-01-10,0,0.486384,0.105385,0.032528,0.001383,0.124977,0.114385,0.9434,0.189394,0.911137,0.088322,0.226471,0.095260,0.9490,0.167453,0.933350,0.081507,0.989435,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-0.397828,-0.028416
5,2020-01-10,0,0.669805,0.000499,0.077567,0.105286,0.117547,0.059100,0.0301,0.152778,0.966825,0.052993,0.007563,0.046235,0.0191,0.154481,0.982847,0.042354,0.959723,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-1.232894,-0.088064
6,2020-01-10,0,0.608664,0.006487,0.062588,0.040884,0.022166,0.041796,0.2469,0.794192,0.861374,0.244603,0.063025,0.029145,0.2426,0.753538,0.872580,0.248461,0.885771,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-61.615090,-4.401078
7,2020-01-10,0,0.588284,0.013474,0.043842,0.023233,0.105901,0.039578,0.4218,1.000000,0.828910,0.303238,0.107143,0.018440,0.4134,1.000000,0.849792,0.302881,0.900627,0.000000,0.000000,0.008627,0.181242,0.0,0.000000,0.076712,0.124614,0.125644,0.019475,0.029914,0.0,0.991758,0.156164,False,False,False,False,False,False,False,False,False,False,False,-20.158358,-1.439883
8,2020-01-17,0,0.444233,0.158009,0.079570,0.001956,0.090529,0.202857,0.9518,0.113636,0.890047,0.080226,0.228151,0.189620,0.9531,0.104953,0.905905,0.079045,0.989766,0.004325,0.013264,0.000000,0.171189,0.0,0.000000,0.095890,0.120494,0.124614,0.000151,0.000000,0.0,0.293956,0.175342,False,False,False,False,False,False,False,False,False,False,False,-1.401711,-0.100122
9,2020-01-17,0,0.483588,0.110375,0.076611,0.004246,0.086535,0.170379,0.9040,0.215909,0.841469,0.1

In [55]:
# import pandas as pd
# from sklearn.preprocessing import MinMaxScaler

# def load_data(file_path):
#     df = pd.read_csv(file_path)
#     df['date'] = pd.to_datetime(df['date'])
#     return df

# def create_condensed_df(df):
#     columns_to_keep = [
#         'date', 'relative_strike', 'mid_call', 'mid_put', 'delta_call', 'delta_put',
#         'gamma_call', 'gamma_put', 'theta_call', 'theta_put', 'vega_call', 'vega_put',
#         'rho_call', 'rho_put', 'hv_current', 'hv_week_ago', 'hv_month_ago', 'hv_year_high',
#         'hv_year_low', 'hv_year_high_length_ago', 'hv_year_low_date_length_ago', 'iv_current',
#         'iv_week_ago', 'iv_month_ago', 'iv_year_high', 'iv_year_low', 'iv_year_high_length_ago',
#         'iv_year_low_date_length_ago', 'delta_neutral_long_straddle_returns', 'mean_daily_delta_neutral_long_straddle_returns'
#     ]

#     df_condensed = df[columns_to_keep].copy()
#     df_condensed['mid_strike_price'] = df['mid_call'] + df['mid_put']
#     df_condensed['delta_straddle'] = df['delta_call'] + df['delta_put']
#     df_condensed['gamma_straddle'] = df['gamma_call'] + df['gamma_put']
#     df_condensed['theta_straddle'] = df['theta_call'] + df['theta_put']
#     df_condensed['vega_straddle'] = df['vega_call'] + df['vega_put']
#     df_condensed['rho_straddle'] = df['rho_call'] + df['rho_put']

#     columns_to_drop = ['mid_call', 'mid_put', 'delta_call', 'delta_put', 'gamma_call', 'gamma_put',
#                        'theta_call', 'theta_put', 'vega_call', 'vega_put', 'rho_call', 'rho_put']

#     return df_condensed.drop(columns=columns_to_drop)

# def normalize_df(df, exclude_columns):
#     columns_to_normalize = [col for col in df.columns if col not in exclude_columns]
#     scaler = MinMaxScaler()
#     df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
#     return df

# def save_csv(df, file_path):
#     df.to_csv(file_path, index=False)

# # Paths
# input_file_path = 'final_straddle_data_raw.csv'
# condensed_file_path = 'final_straddle_data_raw_condensed.csv'
# normalized_file_path = 'final_straddle_data_normalized_condensed.csv'

# # Process
# df_raw = load_data(input_file_path)
# df_condensed = create_condensed_df(df_raw)
# save_csv(df_condensed, condensed_file_path)

# exclude_columns = ['date', 'delta_neutral_long_straddle_returns', 'mean_daily_delta_neutral_long_straddle_returns']
# df_normalized = normalize_df(df_condensed, exclude_columns)
# save_csv(df_normalized, normalized_file_path)
